In [74]:
import copy
import pandas as pd
import numpy as np
import Levenshtein
from tqdm import tqdm

In [75]:
! pwd

/home/ishanu/ZED/Research/emergenet/paper_data_old/irat_qnet/results/animal_predictions


In [76]:
# parameters
threshold=6  # draw tree above this emergenec risk threshold
OUTPUT_DIR='./'
PHYLO_TREE_DIR='./'
PHYLO_DIR='./'
COMBINED_RESULTS='../../../../paper_data_v2/irat_enet/results/animal_predictions/combined_results.csv'
VERBOSE=False

In [77]:
import dill as pickle
with open('tree_collapsed.pkl', 'rb') as file:
    ltree_collapsed = pickle.load(file)
with open('ALLinfo.pkl', 'rb') as file:
    ALLinfoHA = pickle.load(file)    
df=pd.read_csv(COMBINED_RESULTS,index_col=0).reset_index()

In [78]:
df

,id,subtype,ha_accession,na_accession,ha,na,ha_risk,na_risk,geometric_mean_risk,emergence_risk,emergence_risk_var,impact_risk,impact_risk_var
0,A/swine/Missouri/A02524459/2020,H1N1,EPI1765551,EPI1765552,MKAILVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,MNANQRIIAIGTVCMIVGIISLLLQIGNIVSLWISHSIQTGWENHT...,"[0.014528550015281423, 0.014546310217342569, 0...","[0.029578732921608532, 0.02932902190208756, 0....","[0.020730077198125453, 0.020655000628395058, 0...",6.420763,8.045648e-06,6.449227,6.178046e-06
1,A/swine/Ohio/A02524448/2020,H1N1,EPI1765553,EPI1765554,MKAVLVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,MNTNQRIITIGTVCLIVGIISLLLQIGNIVSLWVSHSIQTRWENHT...,"[0.019129440067524317, 0.019127869774293836, 0...","[0.030817728728596058, 0.03059500648117531, 0....","[0.02428015433910791, 0.02419126494657932, 0.0...",6.350109,1.971105e-05,6.387369,1.508262e-05
2,A/swine/South_Dakota/A02524453/2020,H1N1,EPI1765555,EPI1765556,MKAILVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,MNTNQRIITIGTVCMIVGIISLLLQIGNIVSLWISHSIQIEWENHT...,"[0.015332923170890989, 0.015282596391980905, 0...","[0.030647302962040043, 0.030735514122289354, 0...","[0.02167747083291729, 0.02167298911620124, 0.0...",6.401165,1.234722e-05,6.432058,9.471705e-06
3,A/swine/Indiana/A02524458/2020,H1N1,EPI1765559,EPI1765560,MKAVLVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,MNTNQRIITIGTVCLIVGIISLLLQIGNIVSLWVSHSIQTRWENHT...,"[0.01911712558486812, 0.019116256253057823, 0....","[0.030817728728596058, 0.03059500648117531, 0....","[0.024272337966190424, 0.024183919946073922, 0...",6.350272,1.976990e-05,6.387512,1.512778e-05
4,A/swine/Iowa/A02479424/2020,H1N1,EPI1766606,EPI1766607,MKAVLVVMLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,MNTNQRIITIGTVCLIVGIISLLLQIGNIVSLWVSHSIQTRWENHT...,"[0.016496181614260582, 0.01642582283626936, 0....","[0.03411863661430806, 0.03386191944444978, 0.0...","[0.023723979978506695, 0.02358410247795232, 0....",6.360128,1.145076e-05,6.396134,8.766851e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6061,A/chicken/Guangdong/F94/2022,H9N2,EPI2167482,EPI2167484,MKTVSLITILLAATVSNADKICIGYQSTNSTETVDTLTENNVPVTH...,MNPNQKITAIGSVSLIIAIICLLMQIAILTTTMTLHFGQKECSNPL...,"[0.021926223328056015, 0.021926223328056015, 0...","[0.07341886162130176, 0.07341886162130176, 0.0...","[0.04012229251177334, 0.04012229251177334, 0.0...",6.033752,7.888609e-31,6.111792,0.000000e+00
6062,A/chicken/Guangdong/F99/2022,H9N2,EPI2167503,EPI2167506,MEAVSLITILLAATVSNADKICIGYQSTNSTETVDTLTENNVPVTH...,MNPNQKITAIGSVSLIIAIICLLMQIAILTTTMTLHFRQKECSNPS...,"[0.01898769074353181, 0.01898769074353181, 0.0...","[0.05202795688766431, 0.05202795688766431, 0.0...","[0.03143072947611582, 0.03143072947611582, 0.0...",6.202124,0.000000e+00,6.258162,7.888609e-31
6063,A/chicken/Egypt/Menoufia/2021,H9N2,EPI2175832,EPI2175833,MEVIPLMIMLLLVTTNNADKICIGHQSTNSTETVDTLTETGVPVTH...,MNPNQRIIALGSASLTVATICLLIQIAILATTMTLHFKQNEYANTS...,"[0.05534613906024509, 0.05534613906024509, 0.0...","[0.4610969543711347, 0.4610969543711347, 0.461...","[0.15974960455938633, 0.15974960455938633, 0.1...",4.344723,0.000000e+00,4.715680,0.000000e+00
6064,A/chicken/South_Korea/SL20/2020,H9N2,EPI2176583,EPI2176585,METVSLITILVVATVSSADKICIGYQSTNSTETVDTLTENNVPVTH...,MNPNQKITAIGSVSLIIAIICLLMQIAILTTTMTFHFGQKECSNPS...,"[0.02999852644258515, 0.02999852644258515, 0.0...","[0.0512932924280397, 0.0512932924280397, 0.051...","[0.03922656229176604, 0.03922656229176604, 0.0...",6.050818,0.000000e+00,6.126595,0.000000e+00


In [79]:
count=0
Subtype={'H1N1':0,'H3N2':0,'H7N9':0,'H9N2':0}
MaxriskStrain={'H1N1':None,'H3N2':None,'H7N9':None,'H9N2':None}
Subtype_strat={ 6.0:{'H1N1':0,'H3N2':0,'H7N9':0,'H9N2':0},  6.2:{'H1N1':0,'H3N2':0,'H7N9':0,'H9N2':0},              6.3:{'H1N1':0,'H3N2':0,'H7N9':0,'H9N2':0},
               6.4:{'H1N1':0,'H3N2':0,'H7N9':0,'H9N2':0},6.5:{'H1N1':0,'H3N2':0,'H7N9':0,'H9N2':0}}
for node in ltree_collapsed:
    Subtype[node.subtype]=Subtype[node.subtype]+1
    if MaxriskStrain[node.subtype] is None:
        MaxriskStrain[node.subtype]=(ALLinfoHA.seq_infos[node.name.split()[0]].name,
                                     ALLinfoHA.seq_infos[node.name.split()[0]].erisk)
    else:
        if MaxriskStrain[node.subtype][1]<ALLinfoHA.seq_infos[node.name.split()[0]].erisk:
            MaxriskStrain[node.subtype]=(ALLinfoHA.seq_infos[node.name.split()[0]].name,
                                         ALLinfoHA.seq_infos[node.name.split()[0]].erisk)
    for r in [6.0, 6.2,6.3,6.4,6.5]:
        if r<ALLinfoHA.seq_infos[node.name.split()[0]].erisk:
            #print(node.subtype,ALLinfoHA.seq_infos[node.name.split()[0]].erisk)
            Subtype_strat[r][node.subtype]=Subtype_strat[r][node.subtype]+1
        
    count=count+1
maxriskdf = pd.DataFrame(MaxriskStrain)
Subtype_strat_df = pd.DataFrame(Subtype_strat)
display(maxriskdf)
display(Subtype_strat_df)
print(count)
df[df.id.isin(maxriskdf.iloc[0,:].values)]

,H1N1,H3N2,H7N9,H9N2
0,A/swine/Missouri/A02524711/2020,A/swine/Indiana/A02524710/2020,A/Camel/Inner_Mongolia/XL/2020,A/mink/China/chick_embryo/2020
1,6.782248,6.729258,6.704867,6.27162


,6.0,6.2,6.3,6.4,6.5
H1N1,64,57,54,11,4
H3N2,17,17,15,8,4
H7N9,1,1,1,1,1
H9N2,2,2,0,0,0


84


,id,subtype,ha_accession,na_accession,ha,na,ha_risk,na_risk,geometric_mean_risk,emergence_risk,emergence_risk_var,impact_risk,impact_risk_var
218,A/swine/Missouri/A02524711/2020,H1N1,EPI1818121,EPI1818122,MKAILVVMLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,MNPNQKIITIGSICMTIGTANLILQIGNIISIWVSHSIQIGNQSQI...,"[0.007702821053332326, 0.00794167832708008, 0....","[0.0016000691389129699, 0.0019043683598783635,...","[0.0035107045233124007, 0.003888943420830784, ...",6.782248,0.000006,6.767254,0.000005
1902,A/swine/Indiana/A02524710/2020,H3N2,EPI1818137,EPI1818138,MKTIIALSYILCLVFAQKIPGNDNGTATLCLGHHAVPNGTIVKTIT...,MNPNQKIITIGSVSLTISTICFFMQIAILITTIALHFKQYEFNSPP...,"[0.008423100311281287, 0.0044835948774065975, ...","[0.006596476585994605, 0.006619358238135353, 0...","[0.00745404480700585, 0.00544779961894918, 0.0...",6.729258,0.000613,6.720489,0.000476
5814,A/Camel/Inner_Mongolia/XL/2020,H7N9,EPI2026200,EPI2026202,MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTEREVEVVN...,MNPNQKILCTSATAITIGAIAVLIGIANLGLNIGLHLKPGCNCSHS...,"[0.025061424930009856, 0.01711340116551936, 0....","[0.0011227960579890436, 0.0012230115569459082,...","[0.0053046082908169015, 0.004574919387714034, ...",6.704867,0.004832,6.699016,0.003755
6049,A/mink/China/chick_embryo/2020,H9N2,EPI2161544,EPI2161548,METVSLITILLAATVSNADKICIGYQSSNSTETVDTLTENNVPVTH...,MNPNQKITAIGSVSLIIAIICLLMQIAILTMTMTLHFGQKECSNPS...,"[0.016907786181883842, 0.016907786181883842, 0...","[0.04614861940588559, 0.04614861940588559, 0.0...","[0.027933331156592276, 0.027933331156592276, 0...",6.271620,0.000000,6.318778,0.000000


In [80]:
dcf=100*(Subtype_strat_df/Subtype_strat_df.sum(axis=0))

df_=Subtype_strat_df
dcfs=' ('+dcf.round(2).astype(str)+'\%)'
df__=df_.astype(str)+dcfs

df__.columns=df__.columns.astype(str)
df__.index.name='subtype'

In [81]:
from zedstat.textable import textable
textable(df__,
         tabname='../../../../tex/overleaf/Figures/tabdata/riskycount_upd.tex',
         FORMAT='%s',INDEX=True,
         TABFORMAT='L{.4in}|L{.6in}|L{.6in}|L{.6in}|L{.6in}|L{.6in}',LNTERM='\\\\\n')
! cat ../../../../tex/overleaf/Figures/tabdata/riskycount_upd.tex

\begin{tabular}{L{.4in}|L{.6in}|L{.6in}|L{.6in}|L{.6in}|L{.6in}}\hline
subtype&6.0&6.2&6.3&6.4&6.5\\
H1N1& 64  (76.19\%) & 57  (74.03\%) & 54  (77.14\%) & 11  (55.0\%) & 4  (44.44\%) \\
H3N2& 17  (20.24\%) & 17  (22.08\%) & 15  (21.43\%) & 8  (40.0\%) & 4  (44.44\%) \\
H7N9& 1  (1.19\%) & 1  (1.3\%) & 1  (1.43\%) & 1  (5.0\%) & 1  (11.11\%) \\
H9N2& 2  (2.38\%) & 2  (2.6\%) & 0  (0.0\%) & 0  (0.0\%) & 0  (0.0\%) \\
\hline\end{tabular}


In [82]:
def write_fasta(seqs, fasta_file, wrap=80):
    """Write sequences to a fasta file.

    Parameters
    ----------
    seqs : dict[seq_id] -> seq
        Sequences indexed by sequence id.
    fasta_file : str
        Path to write the sequences to.
    wrap: int
        Number of AA/NT before the line is wrapped.
    """
    with open(fasta_file, 'w') as f:
        for gid, gseq in seqs.items():
            f.write('>{}\n'.format(gid))
            for i in range(0, len(gseq), wrap):
                f.write('{}\n'.format(gseq[i:i + wrap])) 


In [88]:
df_top=df[df.id.isin(maxriskdf.iloc[0,:].values)]
# pick teh top H1N1 and H3N2
#df_top=df_top[df_top.subtype.isin(['H1N1','H3N2'])]
df_top=df_top[df_top.subtype.isin(['H3N2'])]


In [89]:
RFILES={name+'('+subtype+')':i for i,name,subtype in zip(df_top.ha.values,
                 df_top.ha_accession.values,
                 df_top.subtype.values)}

In [90]:
FASTANAME='../../../../tex/overleaf/Figures/plotdata/risky4_domtop.fasta'
FASTANAME='../../../../tex/overleaf/Figures/plotdata/risky4_h3n2.fasta'
#write_fasta(RFILES, fasta_file=FASTANAME)

In [91]:
#! cat ../../../../tex/overleaf/Figures/plotdata/risky4.fasta

In [102]:
human_dom_seq_h1n1_north='../../../../paper_data_v2/enet_predictions/dominant_sequences/results/north_h1n1_ha/north_h1n1_ha_21_22/dom_seqs.csv'
human_dom_seq_h3n2_north='../../../../paper_data_v2/enet_predictions/dominant_sequences/results/north_h3n2_ha/north_h3n2_ha_21_22/dom_seqs.csv'
domf=pd.concat([pd.read_csv(human_dom_seq_h1n1_north,index_col=0).assign(subtype='H1N1',protein='HA').head(3),
               pd.read_csv(human_dom_seq_h3n2_north,index_col=0).assign(subtype='H3N2',protein='HA').head(3)])
#domf=domf.assign(subtype=[x.split('_')[1].upper() for x in domf.index.values],protein=[x.split('_')[2].upper() for x in domf.index.values])
domf=domf.tail(3)
domf

,name,date,sequence,acc_na,sequence_na,cluster_size,subtype,protein
acc,,,,,,,,
EPI1949137,A/Baltimore/JH/001/2021,2021-09-11,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,EPI1949136,MNPNQKIITIGSVSLTISTICFFMQIAILITTVTLHFKQYEFNSPP...,2751,H3N2,HA
EPI1998573,A/Myanmar/I026/2021,2021-08-01,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,EPI1998572,MNPNQKIITIGSVSLTISTICFFMQIAILITTVTLHFKQYEFNSPP...,2386,H3N2,HA
EPI1859978,A/Darwin/12/2021,2021-02-21,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,EPI1859977,MNPNQKIITIGSVSLTISTICFFMQIAILITTVTLHFKQYEFNSPP...,787,H3N2,HA


In [103]:
DOMFILES={'HUM'+'('+name[:5]+'|'+subtype+')':i for i,name,subtype in zip(domf.sequence.values,
                 domf.name.values,
                 domf.subtype.values)}
DOMFILES

{'HUM(A/Bal|H3N2)': 'MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGEICGSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSNCYPYDVPGYASLRSLVASSGTLEFKNESFNWTGVKQNGTSSACIRGSSSSFFSRLNWLTSINNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQISLFAQSSGRITVSTKRSQQAVIPNIGSRPRIRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRNGKSSIMRSDAPIGRCKSECITPNGSIPNDKPFQNVNRITYGACPRYVKQSTLKLATGMRNVPEKQTRGIFGAIAGFIENGWEGMVDGWYGFRHQNSEGRGQAADLKSTQAAIDQINGKLNRLIGKTNEKFHQIEKEFSEVEGRVQDLEKYVEDTKIDLWSYNAELLVALENQHTIDLTDSEMNKLFEKTKKQLRENAEDMGNGCFKIYHKCDNACIGSIRNETYDHNVYRDEALNNRFQIKGVELKSGYKDWILWISFAMSCFLLCIALLGFIMWACQKGNIRCNIC',
 'HUM(A/Mya|H3N2)': 'MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGEICNSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSSCYPYDVPDYASLRSLVASSGTLEFKNESFNWTGVKQNGTSSACIRGSSSSFFSRLNWLTSLNNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQFSLFAQSSGRITVSTKRSQQAVIPNIGSRPRIRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRSGKSSIMRSDAPIGKCKSECITPNGSIPNDKPFQNVNRITYGACPRYVKQSTLKLATGMRNVPEKQTRGIFGAIAGFIENGWEGMVDGWYGFRHQNSEGRGQAADLKSTQAAI

In [104]:
DOMFILES.update(RFILES)
DOMFILES

{'HUM(A/Bal|H3N2)': 'MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGEICGSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSNCYPYDVPGYASLRSLVASSGTLEFKNESFNWTGVKQNGTSSACIRGSSSSFFSRLNWLTSINNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQISLFAQSSGRITVSTKRSQQAVIPNIGSRPRIRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRNGKSSIMRSDAPIGRCKSECITPNGSIPNDKPFQNVNRITYGACPRYVKQSTLKLATGMRNVPEKQTRGIFGAIAGFIENGWEGMVDGWYGFRHQNSEGRGQAADLKSTQAAIDQINGKLNRLIGKTNEKFHQIEKEFSEVEGRVQDLEKYVEDTKIDLWSYNAELLVALENQHTIDLTDSEMNKLFEKTKKQLRENAEDMGNGCFKIYHKCDNACIGSIRNETYDHNVYRDEALNNRFQIKGVELKSGYKDWILWISFAMSCFLLCIALLGFIMWACQKGNIRCNIC',
 'HUM(A/Mya|H3N2)': 'MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGEICNSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSSCYPYDVPDYASLRSLVASSGTLEFKNESFNWTGVKQNGTSSACIRGSSSSFFSRLNWLTSLNNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQFSLFAQSSGRITVSTKRSQQAVIPNIGSRPRIRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRSGKSSIMRSDAPIGKCKSECITPNGSIPNDKPFQNVNRITYGACPRYVKQSTLKLATGMRNVPEKQTRGIFGAIAGFIENGWEGMVDGWYGFRHQNSEGRGQAADLKSTQAAI

In [105]:
write_fasta(DOMFILES,FASTANAME)

In [96]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW
my_query = SeqIO.read("EPI1818121.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastp", "nr", my_query.seq)
blast_result = open("my_blast.xml", "w")
blast_result.write(result_handle.read())
blast_result.close()
result_handle.close()

In [ ]:
def parse_blast(resultfile): #takes in the BLAST result, outputs list that can be made into csv
    from Bio.Blast import NCBIXML
    result_handle = open(resultfile)
    blast_records = NCBIXML.parse(result_handle)
    csv_list = []
    
    header = [  'accession','Query',
                'Name', 'Length', 'Score', 'Expect',
                'QueryStart', 'QueryEnd',
                'SubjectStart', 'SubjectEnd','pct'
            ]
    
    #csv_list.append(header)
    count = 0
    for blast_record in blast_records:
        '''help(blast_record.alignments[0].hsps[0])''' # these give help info for the parts 
        '''help(blast_record.alignments[0])        '''
        count +=1
        
        query = blast_record.query
        for alignment in blast_record.alignments:

            name = alignment.title
            length = alignment.length
    
            hsp = alignment.hsps[0] # I don't know if we will ever have more than one, so might as well take the first one.
            score = hsp.score
            expect = hsp.expect
            querystart = hsp.query_start
            queryend = hsp.query_end
            subjectstart = hsp.sbjct_start
            subjectend = hsp.sbjct_end
            pct=hsp.positives/hsp.align_length
            accession=alignment.accession
            row = [accession,query,name,length,score,expect,querystart,queryend,subjectstart,subjectend,pct]
            csv_list.append(row)
            
    result_handle.close()
    return pd.DataFrame(csv_list,columns=header)
 

In [ ]:
csv_list=parse_blast('my_blast.xml')
csv_list

In [ ]:
from Bio import Entrez
Entrez.email = "ishanu@uchicago.edu"     # Always tell NCBI who you are
handle = Entrez.esearch(db="protein", term="QJT24340")
record = Entrez.read(handle)
id=record["IdList"]
id

In [ ]:
handle = Entrez.efetch(db="protein", rettype="gb", retmode="text", id="1834373412")
#record = SeqIO.read(handle, "genbank")
handle.read()

H9N2: minks are probable mixing vessels
https://www.ncbi.nlm.nih.gov/nuccore/ON870694.1
 https://www.tandfonline.com/doi/full/10.1080/22221751.2021.1899058
 Pandemic influenza, typically caused by the reassortment of human and avian influenza viruses, can result in severe or fatal infections in humans. Timely identification of potential pandemic viruses must be a priority in influenza virus surveillance. However, the range of host species responsible for the generation of novel pandemic influenza viruses remains unclear. In this study, we conducted serological surveys for avian and human influenza virus infections in farmed mink and determined the susceptibility of mink to prevailing avian and human virus subtypes. The results showed that farmed mink were commonly infected with human (H3N2 and H1N1/pdm) and avian (H7N9, H5N6, and H9N2) influenza A viruses. Correlational analysis indicated that transmission of human influenza viruses occurred from humans to mink, and that feed source was a probable route of avian influenza virus transmission to farmed mink. Animal experiments showed that mink were susceptible and permissive to circulating avian and human influenza viruses, and that human influenza viruses (H3N2 and H1N1/pdm), but not avian viruses, were capable of aerosol transmission among mink. These results indicate that farmed mink could be highly permissive “mixing vessels” for the reassortment of circulating human and avian influenza viruses. 
 
 H7N9
 https://www.bmj.com/content/347/bmj.f4752?tab=responses
 HH transmission has been suspected
 Asian lineage H7N9 virus is rated by the Influenza Risk Assessment Tool as having the greatest potential to cause a pandemic, as well as potentially posing the greatest risk to severely impact public health if it were to achieve sustained human-to-human transmission.
 
 H3N2 swine
 https://www.cdc.gov/flu/swineflu/spotlights/first-human-infection-2022.htm
 Human infection detected in US this Aug
 
 H1N1
 Submitted (03-NOV-2020) USDA Swine Surveillance, USDA Swine
            Surveillance, 1920 Dayton, Ames, IA 50010, USA
COMMENT     Method: conceptual translation.

